#EASYMONEY

In [1]:
from IPython.display import Image
Image(url='https://lh3.googleusercontent.com/LzCgpXQ_vD9O717mlEV9wzRhhfXVqn8OYnd2guu3wz8bwrlt3B6C7rTk4gwvIO12yJccnayagm8qbiNQlbViixL4n1deIwPTIj79LdgnkDG7RQEfAEY97REmTyVAQZVrCpFr_Wqvh8fmCglKry-Cglj3kDYNIZCHill7ela-X9hYWUvQy4XKJ0vN3LGsgfoJNVqnsd9l9aTIxn6w2I-22k0OEXnAWmrG6zifbUhy4cn097Ub-AqTS_qIivePD3V-SdBKWjNzRT7GcXIWZe3_4dtiIJhbCx_curxwwGfMDNYkuK4BhXmjCzLEeyDpfZHJEoMdUNOp_EyPPPpl2TnHnuGABzyDd0sonYmmhywApURtdQLcyEehzOIg64YxotQg7JBSykfXLbEEYtBaAWz_VpaDF7g73hbJEGxx51Kk5pZZNLvSS8wE9BehEIxv_UQ1ohIzkdY1gNb6-5RoWGvLkOe0KYjTmGopQ7J_1HJCQuqLjZq6UTIkv2pdmCrtuiwef5xAu8JYOQnU6TQCytuMBO9tD3MbpRLCCxGpJNiYEIQ9YNsb7ooRhRrhOR4aneAcTsCeDBGYSz5E9lgw0lHMVi0vHVF79RXorsW0YC5Ed91J1Bd5sHB78gkX4gwBuKa-fUOfHvevPyXu8BPM0etS4ziaxNDaCJSp48Lohpo5XDBfabRF0Tt81CTrtKbGVJGAulSmo-UmWpJI2HthBpL3GR_yhdG6pJ2uVfTuPMjOqU9zPiWpL7YAoKWdRdeqdca7ZFtcZU6ri_cYgIP6bOadYiEXm5daUXjlr5iRE11z-bD1vZMi-J6Qm-r9SPVXHIgIy8CRuNIJBC5UBTvj5aXrHdWtqHOb0uhftq5GtrpKMsVAVIU0w1HES-YHrW7mjrbMDbFUpSo1hWQAYwV44Cb5BKghaRQEj9Tp6nq9jhYbKfE=w566-h262-no?authuser=0')  

#IMPORTAMOS LIBRERIAS

In [2]:
import pandas as pd 
import numpy as np

In [3]:
pd.options.display.max_rows=25
pd.options.display.max_columns=15

In [4]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder
#from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer, SimpleImputer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
##from sklearn.compose import ColumnTransformer

In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import time

#FUNCIONES

In [8]:
def OHE(dataframe, column_name):
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    dataframe = pd.concat([dataframe, _dummy_dataset], axis=1)
    return dataframe.drop(column_name, axis=1)

In [9]:
def setGrouped(dataframe, column, num_values,nombre_grupo):
    top_categories = dataframe[column].value_counts().head(num_values)
    top_categories_list = top_categories.index.to_list()
    top_categories_list.append(nombre_grupo)
    dataframe[column]= np.where(~dataframe[column].isin(top_categories_list),nombre_grupo,dataframe[column])
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    return dataframe[column]

####La función al llamarla agrupará todos los valores con una frecuencia inferior a la definida en un único valor

#IMPORTAMOS DATOS

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
dfc = pd.read_csv("/content/drive/MyDrive/TFM/data/commercial_activity_df.csv")

In [12]:
dfp = pd.read_csv("/content/drive/MyDrive/TFM/data/products_df.csv")

In [13]:
dfsd = pd.read_csv("/content/drive/MyDrive/TFM/data/sociodemographic_df.csv")

In [14]:
#Creamos esta variable por si fuera necesario para operar en vez de usar HOY para que cuadre con las fechas del "roleplay"
HOY = pd.to_datetime('2019-06-01')

Tenemos tres datasets distintos con aspectos distintos del negocio. Procesaremos los tres por separado, incluyendo la creación de nuevas variables. A partir de estos datasets ya preparados, exportaremos nuevos datasets enfocados a cada tarea específica para facilitar el trabajo.

#PROCESAMIENTO DE DATASETS DE PARTIDA Y 
#GENERACIÓN DE NUEVOS DATASETS PARA ANÁLISIS

##1 DATOS DEMOGRÁFICOS DE LOS USUARIOS

Tales como:

- Sexo
- Edad 
- Segmento
- Salario 
- De dónde son 

Partimos del dfsd

*(SD1)*

In [15]:
SD1=dfsd.copy()

In [16]:
SD1.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [17]:
dfc.head()

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
3,3,1050613,2018-01-28,2015-08-10,KHD,0.0,03 - UNIVERSITARIO
4,4,1050614,2018-01-28,2015-08-10,KHE,1.0,03 - UNIVERSITARIO


In [18]:
SD1['SEGMENTO']=dfc['segment']

In [19]:
SD1.drop('Unnamed: 0', axis=1, inplace = True)

In [20]:
SD1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   pk_cid        int64  
 1   pk_partition  object 
 2   country_id    object 
 3   region_code   float64
 4   gender        object 
 5   age           int64  
 6   deceased      object 
 7   salary        float64
 8   SEGMENTO      object 
dtypes: float64(2), int64(2), object(5)
memory usage: 409.4+ MB


In [21]:
SD1.isnull().sum()

pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1512103
SEGMENTO         133944
dtype: int64

In [22]:
SD1['region_code'].fillna(value='??', inplace=True)
SD1['gender'].fillna(value='ND', inplace=True)
SD1['salary'].fillna(value=0, inplace=True)
SD1['pk_partition']=pd.to_datetime(SD1['pk_partition'])
SD1['SEGMENTO'].fillna(value='04 - OTROS', inplace=True)
SD1['gender']= np.where(SD1['gender'] == 'H', 'M',SD1['gender'])
SD1['gender']= np.where(SD1['gender'] == 'V', 'H',SD1['gender'])

In [23]:
SD1.columns

Index(['pk_cid', 'pk_partition', 'country_id', 'region_code', 'gender', 'age',
       'deceased', 'salary', 'SEGMENTO'],
      dtype='object')

In [24]:
SD1_columns_friend = ['ID_USR','FECHA_PARTICION','PAIS','REGION(ESP)','SEXO','EDAD','FALLECIDO/A','SALARIO','SEGMENTO']

In [25]:
SD1.columns=SD1_columns_friend

In [26]:
SD1['SALARIO'].max()

28894395.51

In [27]:
SD1['RANGO_SALARIAL'] = pd.cut(x=SD1['SALARIO'], bins=[0,50000,100000,150000,200000,30000000],
                    labels=['0 a 50.000€', '50.000 a 100.000€', '100.000 a 150.000€',
                            '150.000 a 200.000€', 'más de 200.000 €'])

In [28]:
SD1['RANGO_SALARIAL'].fillna(value='0 a 50.000€', inplace=True)

In [29]:
SD1['DE_ESPAÑA']=np.where(SD1['PAIS']=='ES','SI','NO')

In [30]:
SD1['DE_ESPAÑA'].value_counts(normalize=True)

SI    0.999622
NO    0.000378
Name: DE_ESPAÑA, dtype: float64

In [31]:
SD1['REGION(ESP)'].nunique()

53

In [32]:
SD1['REGION(ESP)'].value_counts().head(3)

28.0    1185757
8.0      600362
46.0     358142
Name: REGION(ESP), dtype: int64

In [33]:
SD1['REGION(ESP)'].value_counts().tail(3)

51.0    2896
52.0    2638
??      2264
Name: REGION(ESP), dtype: int64

Con toda probabilidad es el código de provincia del INE, ya que coinciden los tres valores más habituales con las tres provincias más pobladas (Madrid, Barcelona y Valencia) y los dos que menos (descartando los nulos) con Ceuta y Melilla

https://www.ine.es/daco/daco42/codmun/cod_ccaa_provincia.htm



In [34]:
CCAA = { 4.0 : 'ANDALUCIA' ,11.0:'ANDALUCIA',14.0:'ANDALUCIA',18.0:'ANDALUCIA',21.0:'ANDALUCIA',23.0:'ANDALUCIA', 29.0 : 'ANDALUCIA' ,41.0:'ANDALUCIA',
        22.0:'ARAGON',44.0:'ARAGON',50.0:'ARAGON',
        33.0:'ASTURIAS',
        7.0:'BALEARES', 
        35.0:'CANARIAS',38.0:'CANARIAS',
        39.0:'CANTABRIA',
        5.0:'CASTILLA Y LEON',9.0:'CASTILLA Y LEON',24.0:'CASTILLA Y LEON',34.0:'CASTILLA Y LEON',37.0:'CASTILLA Y LEON',40.0:'CASTILLA Y LEON',42.0:'CASTILLA Y LEON',47.0:'CASTILLA Y LEON',49.0:'CASTILLA Y LEON',
        2.0:'CASTILLA LA MANCHA',13.0:'CASTILLA LA MANCHA',16.0:'CASTILLA LA MANCHA',19.0:'CASTILLA LA MANCHA',45.0:'CASTILLA LA MANCHA',
        8.0:'CATALUÑA',17.0:'CATALUÑA',25.0:'CATALUÑA',43.0:'CATALUÑA',
        3.0:'CDAD. VALENCIANA',12.0:'CDAD. VALENCIANA',46.0:'CDAD. VALENCIANA',
        6.0:'EXTREMADURA',10.0:'EXTREMADURA',
        15.0:'GALICIA',27.0:'GALICIA',32.0:'GALICIA',36.0:'GALICIA',
        28.0:'MADRID',
        30.0:'MURCIA',
        31.0:'NAVARRA',
        1.0:'PAIS VASCO',48.0:'PAIS VASCO',20.0:'PAIS VASCO',26.0:'PAIS VASCO',
        51.0:'CEUTA Y MELILLA',52.0:'CEUTA Y MELILLA',
        26.0:'LA RIOJA',
        '??':'NO APLICA/DESCONOCIDA'}

In [35]:
SD1['COM.AUTONOMA_ESP']=SD1['REGION(ESP)'].map(CCAA)

In [36]:
SD1['COM.AUTONOMA_ESP'].value_counts()

MADRID                   1185757
ANDALUCIA                 953118
CATALUÑA                  749780
CDAD. VALENCIANA          598209
GALICIA                   538163
CASTILLA Y LEON           402626
CASTILLA LA MANCHA        307347
MURCIA                    296856
EXTREMADURA               205870
ARAGON                    166711
CANARIAS                  150983
ASTURIAS                  134562
CANTABRIA                  69589
PAIS VASCO                 67088
BALEARES                   62436
LA RIOJA                   34790
NAVARRA                    31241
CEUTA Y MELILLA             5534
NO APLICA/DESCONOCIDA       2264
Name: COM.AUTONOMA_ESP, dtype: int64

In [37]:
SD1['EDAD'].max()

105

In [38]:
#rangueo edades por si es mejor cara a presentación
SD1['RANGO_EDAD'] = pd.cut(x=SD1['EDAD'], bins=[0,18,35,50,65,80,120],
                           labels=['0 a 18', '19 a 35', '36 a 50','51 a 65','66 a 80','> 80'])


In [39]:
SD1['RANGO_EDAD'].value_counts()

19 a 35    4566946
36 a 50     916328
51 a 65     320974
66 a 80      91143
0 a 18       37716
> 80         29817
Name: RANGO_EDAD, dtype: int64

In [40]:
SD1['ID_USR'].nunique()

456373

Versión procesada del dataset sociodemográfico:

In [41]:
SD1.to_csv('SD1.csv')

###1B ANÁLISIS SOCIODEMOGRÁFICO

Generamos otro dataset de la última partición para el análisis de la situación actual 

*(SD1_UR)*

In [42]:
#agrupamos por fecha particion
GB_SD1 = SD1.groupby("FECHA_PARTICION")

In [43]:
#para la foto fija actual, cogeremos los datos de la ultima particion
SD1_UR=GB_SD1.get_group((SD1['FECHA_PARTICION'].max()))

In [44]:
SD1_UR.head()

,ID_USR,FECHA_PARTICION,PAIS,REGION(ESP),SEXO,EDAD,FALLECIDO/A,SALARIO,SEGMENTO,RANGO_SALARIAL,DE_ESPAÑA,COM.AUTONOMA_ESP,RANGO_EDAD
5519929,657826,2019-05-28,ES,25.0,M,44,N,54493.38,02 - PARTICULARES,50.000 a 100.000€,SI,CATALUÑA,36 a 50
5519930,657817,2019-05-28,ES,8.0,H,32,N,0.00,04 - OTROS,0 a 50.000€,SI,CATALUÑA,19 a 35
5519931,657986,2019-05-28,ES,41.0,M,39,N,100993.17,02 - PARTICULARES,100.000 a 150.000€,SI,ANDALUCIA,36 a 50
5519932,657905,2019-05-28,ES,28.0,M,85,N,154059.09,01 - TOP,150.000 a 200.000€,SI,MADRID,> 80
5519933,657336,2019-05-28,ES,28.0,H,38,N,0.00,02 - PARTICULARES,0 a 50.000€,SI,MADRID,36 a 50


In [45]:
SD1_UR['FALLECIDO/A'].value_counts()

N    442909
S        86
Name: FALLECIDO/A, dtype: int64

In [46]:
#Y eliminamos los fallecidos porque ya no están o no deberían estar activos en la app
SD1_UR.drop(SD1_UR[SD1_UR['FALLECIDO/A']=='S'].index, axis=0 , inplace=True)


<ipython-input-46-f6f6d2aa0129>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SD1_UR.drop(SD1_UR[SD1_UR['FALLECIDO/A']=='S'].index, axis=0 , inplace=True)


In [47]:
SD1_UR.drop(['FALLECIDO/A'], axis=1 , inplace=True)

<ipython-input-47-ab99b58402bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SD1_UR.drop(['FALLECIDO/A'], axis=1 , inplace=True)


In [48]:
SD1_UR.to_csv('SD1_UR.csv')


###1C EVOLUCIÓN DATOS SOCIODEMOGRÁFICOS

Generamos otro dataset que muestre como han variado estos datos en el tiempo:

- Cuántos son de España
- Cuántos son hombres y cuántos mujeres
- Edad media
- Cuántos por segmento
- Cuántos en cada com. autónoma
- Salario medio
- Total de fallecidos

*(SD1_EVO)*

In [49]:
SD2=SD1.copy()

In [50]:
SD2.head().T

,0,1,2,3,4
ID_USR,1375586,1050611,1050612,1050613,1050614
FECHA_PARTICION,2018-01-28 00:00:00,2018-01-28 00:00:00,2018-01-28 00:00:00,2018-01-28 00:00:00,2018-01-28 00:00:00
PAIS,ES,ES,ES,ES,ES
REGION(ESP),29.0,13.0,13.0,50.0,50.0
SEXO,M,H,H,M,H
EDAD,35,23,23,22,23
FALLECIDO/A,N,N,N,N,N
SALARIO,87218.1,35548.74,122179.11,119775.54,0.0
SEGMENTO,02 - PARTICULARES,03 - UNIVERSITARIO,03 - UNIVERSITARIO,03 - UNIVERSITARIO,03 - UNIVERSITARIO
RANGO_SALARIAL,50.000 a 100.000€,0 a 50.000€,100.000 a 150.000€,100.000 a 150.000€,0 a 50.000€


In [51]:
columnas_usadas=['PAIS','REGION(ESP)','RANGO_SALARIAL','RANGO_EDAD']
SD2.drop(columnas_usadas, axis=1, inplace = True)

In [52]:
ohe=['SEXO','FALLECIDO/A','SEGMENTO','DE_ESPAÑA','COM.AUTONOMA_ESP']

In [53]:
for column in SD2[ohe].columns:
    SD2 = OHE(SD2, column)

In [54]:
SD1a_EVO=SD2.groupby("FECHA_PARTICION").agg(
    TOTAL_USR=("ID_USR",lambda series: len(series[series >= 0])),
    TOTAL_ESPAÑA = ("DE_ESPAÑA_SI", np.sum),
    TOTAL_MUJERES = ("SEXO_M", np.sum),
    TOTAL_HOMBRES = ("SEXO_H", np.sum),
    TOTAL_FALLECIMIENTOS = ("FALLECIDO/A_S", np.sum),
    TOTAL_ANDALUCIA= ("COM.AUTONOMA_ESP_ANDALUCIA", np.sum),
    TOTAL_ARAGON= ("COM.AUTONOMA_ESP_ARAGON", np.sum),
    TOTAL_ASTURIAS= ("COM.AUTONOMA_ESP_ASTURIAS", np.sum),
    TOTAL_BALEARES= ("COM.AUTONOMA_ESP_BALEARES", np.sum),
    TOTAL_CANARIAS= ("COM.AUTONOMA_ESP_CANARIAS", np.sum),
    TOTAL_CANTABRIA= ("COM.AUTONOMA_ESP_CANTABRIA", np.sum),
    TOTAL_CASTILLA_LA_MANCHA = ("COM.AUTONOMA_ESP_CASTILLA LA MANCHA", np.sum),
    TOTAL_CASTILLA_Y_LEON = ("COM.AUTONOMA_ESP_CASTILLA Y LEON", np.sum),
    TOTAL_CATALUÑA= ("COM.AUTONOMA_ESP_CATALUÑA", np.sum),
    TOTAL_CDAD_VALENCIANA= ("COM.AUTONOMA_ESP_CDAD. VALENCIANA", np.sum),
    TOTAL_CEUTA_Y_MELILLA= ("COM.AUTONOMA_ESP_CEUTA Y MELILLA", np.sum),
    TOTAL_EXTREMADURA= ("COM.AUTONOMA_ESP_EXTREMADURA", np.sum),
    TOTAL_GALICIA= ("COM.AUTONOMA_ESP_GALICIA", np.sum),
    TOTAL_LA_RIOJA= ("COM.AUTONOMA_ESP_LA RIOJA", np.sum),
    TOTAL_MADRID= ("COM.AUTONOMA_ESP_MADRID", np.sum),
    TOTAL_MURCIA= ("COM.AUTONOMA_ESP_MURCIA", np.sum),
    TOTAL_NAVARRA= ("COM.AUTONOMA_ESP_NAVARRA", np.sum),
    TOTAL_PAIS_VASCO= ("COM.AUTONOMA_ESP_PAIS VASCO", np.sum),
    TOTAL_SEGMENTO_TOP = ("SEGMENTO_01 - TOP", np.sum),
    TOTAL_SEGMENTO_PARTICULARES = ("SEGMENTO_02 - PARTICULARES", np.sum),
    TOTAL_SEGMENTO_UNIVERSITARIO = ("SEGMENTO_03 - UNIVERSITARIO", np.sum),
    TOTAL_SEGMENTO_OTROS = ("SEGMENTO_04 - OTROS", np.sum),

).astype(int)

In [55]:
SD1b_EVO=SD2.groupby("FECHA_PARTICION").agg(
    MEDIA_EDAD = ("EDAD", np.mean),
    MEDIA_SALARIO = ("SALARIO", np.mean),
).round(2)

In [56]:
SD1a_EVO

,TOTAL_USR,TOTAL_ESPAÑA,TOTAL_MUJERES,TOTAL_HOMBRES,TOTAL_FALLECIMIENTOS,TOTAL_ANDALUCIA,TOTAL_ARAGON,...,TOTAL_MURCIA,TOTAL_NAVARRA,TOTAL_PAIS_VASCO,TOTAL_SEGMENTO_TOP,TOTAL_SEGMENTO_PARTICULARES,TOTAL_SEGMENTO_UNIVERSITARIO,TOTAL_SEGMENTO_OTROS
FECHA_PARTICION,,,,,,,,,,,,,,,
2018-01-28,239493,239396,125478,114015,47,42315,7019,...,13182,1323,2446,3951,65722,169792,28
2018-02-28,242521,242423,126785,115736,47,42841,7090,...,13276,1337,2538,4220,68211,170064,26
2018-03-28,245258,245157,127937,117320,47,43316,7128,...,13335,1350,2603,4527,70447,170261,23
2018-04-28,247463,247361,128893,118569,46,43690,7163,...,13378,1365,2672,4816,72242,170396,9
2018-05-28,249926,249823,129929,119996,47,44064,7211,...,13449,1380,2735,5131,74236,170551,8
2018-06-28,252104,252002,130856,121247,45,44395,7259,...,13495,1387,2779,5323,76111,170670,0
2018-07-28,339339,339200,174534,164804,55,54631,9182,...,16179,1806,3923,5376,108178,208935,16850
2018-08-28,352922,352781,181898,171022,57,55512,9579,...,16728,1868,4047,5571,112669,220175,14507
2018-09-28,375323,375179,193897,181424,64,57769,10237,...,17844,1972,4218,5760,116549,229676,23338


In [57]:
SD1b_EVO

,MEDIA_EDAD,MEDIA_SALARIO
FECHA_PARTICION,,
2018-01-28,29.29,92605.93
2018-02-28,29.42,92609.89
2018-03-28,29.54,92736.75
2018-04-28,29.64,92763.40
2018-05-28,29.75,92900.34
2018-06-28,29.84,93009.84
2018-07-28,29.89,95920.54
2018-08-28,29.80,95707.69
2018-09-28,29.60,93124.34


In [58]:
SD1_EVO= SD1a_EVO.merge(SD1b_EVO, on='FECHA_PARTICION', how='left')

In [59]:
SD1_EVO

,TOTAL_USR,TOTAL_ESPAÑA,TOTAL_MUJERES,TOTAL_HOMBRES,TOTAL_FALLECIMIENTOS,TOTAL_ANDALUCIA,TOTAL_ARAGON,...,TOTAL_PAIS_VASCO,TOTAL_SEGMENTO_TOP,TOTAL_SEGMENTO_PARTICULARES,TOTAL_SEGMENTO_UNIVERSITARIO,TOTAL_SEGMENTO_OTROS,MEDIA_EDAD,MEDIA_SALARIO
FECHA_PARTICION,,,,,,,,,,,,,,,
2018-01-28,239493,239396,125478,114015,47,42315,7019,...,2446,3951,65722,169792,28,29.29,92605.93
2018-02-28,242521,242423,126785,115736,47,42841,7090,...,2538,4220,68211,170064,26,29.42,92609.89
2018-03-28,245258,245157,127937,117320,47,43316,7128,...,2603,4527,70447,170261,23,29.54,92736.75
2018-04-28,247463,247361,128893,118569,46,43690,7163,...,2672,4816,72242,170396,9,29.64,92763.40
2018-05-28,249926,249823,129929,119996,47,44064,7211,...,2735,5131,74236,170551,8,29.75,92900.34
2018-06-28,252104,252002,130856,121247,45,44395,7259,...,2779,5323,76111,170670,0,29.84,93009.84
2018-07-28,339339,339200,174534,164804,55,54631,9182,...,3923,5376,108178,208935,16850,29.89,95920.54
2018-08-28,352922,352781,181898,171022,57,55512,9579,...,4047,5571,112669,220175,14507,29.80,95707.69
2018-09-28,375323,375179,193897,181424,64,57769,10237,...,4218,5760,116549,229676,23338,29.60,93124.34


In [60]:
SD1_EVO.to_csv('SD1_EVO.csv')

##2 DATOS DE PRODUCTOS CONTRATADOS

Para cada usuario, qué productos tiene en cada partición

Partimos del dfp y añadiremos una variable sobre el total de productos

*(P1)*

In [61]:
dfp.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,...,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,...,0.0,0.0,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,...,0.0,0.0,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,...,0.0,0.0,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,...,0.0,0.0,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,...,0.0,0.0,0,0,0,0,1


In [62]:
P1=dfp.copy()

In [63]:
P1.drop('Unnamed: 0', axis=1, inplace = True)

In [64]:
P1.isnull().sum()

pk_cid                 0
pk_partition           0
short_term_deposit     0
loans                  0
mortgage               0
funds                  0
securities             0
long_term_deposit      0
em_account_pp          0
credit_card            0
payroll               61
pension_plan          61
payroll_account        0
emc_account            0
debit_card             0
em_account_p           0
em_acount              0
dtype: int64

In [65]:
P1.fillna(value=0, inplace=True)

In [66]:
P1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   pk_cid              int64  
 1   pk_partition        object 
 2   short_term_deposit  int64  
 3   loans               int64  
 4   mortgage            int64  
 5   funds               int64  
 6   securities          int64  
 7   long_term_deposit   int64  
 8   em_account_pp       int64  
 9   credit_card         int64  
 10  payroll             float64
 11  pension_plan        float64
 12  payroll_account     int64  
 13  emc_account         int64  
 14  debit_card          int64  
 15  em_account_p        int64  
 16  em_acount           int64  
dtypes: float64(2), int64(14), object(1)
memory usage: 773.4+ MB


In [67]:
P1['payroll'] = P1['payroll'].astype(int)
P1['pension_plan'] = P1['pension_plan'].astype(int)
P1['pk_partition']=pd.to_datetime(P1['pk_partition'])

In [68]:
P1.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount'],
      dtype='object')

In [69]:
columns_friendly=['ID_USR','FECHA_PARTICION','PROD_DEPOSITO_CP','PROD_PRESTAMOS','PROD_HIPOTECA',
                  'PROD_FONDOS','PROD_VALORES','PROD_DEPOSITOS_LP','PROD_CUENTA_EM++',
                  'PROD_TARJETA_CREDITO','PROD_DOMICILIACIONES','PROD_PLAN_PENSIONES','PROD_CUENTA_BON_DOMICILIA',
                  'PROD_CUENTA_EM_CRYPTO','PROD_TARJETA_DEBITO','PROD_CUENTA_EM+','PROD_CUENTA_EM']

In [70]:
P1.columns=columns_friendly

In [71]:
productos=['PROD_DEPOSITO_CP','PROD_PRESTAMOS','PROD_HIPOTECA',
           'PROD_FONDOS','PROD_VALORES','PROD_DEPOSITOS_LP','PROD_CUENTA_EM++',
           'PROD_TARJETA_CREDITO','PROD_DOMICILIACIONES','PROD_PLAN_PENSIONES','PROD_CUENTA_BON_DOMICILIA',
           'PROD_CUENTA_EM_CRYPTO','PROD_TARJETA_DEBITO','PROD_CUENTA_EM+','PROD_CUENTA_EM']

In [72]:
P1['TOTAL_PROD']=P1[productos].sum(axis=1)

In [73]:
P1.head()

,ID_USR,FECHA_PARTICION,PROD_DEPOSITO_CP,PROD_PRESTAMOS,PROD_HIPOTECA,PROD_FONDOS,PROD_VALORES,...,PROD_PLAN_PENSIONES,PROD_CUENTA_BON_DOMICILIA,PROD_CUENTA_EM_CRYPTO,PROD_TARJETA_DEBITO,PROD_CUENTA_EM+,PROD_CUENTA_EM,TOTAL_PROD
0,1375586,2018-01-28,0,0,0,0,0,...,0,0,0,0,0,1,1
1,1050611,2018-01-28,0,0,0,0,0,...,0,0,0,0,0,1,1
2,1050612,2018-01-28,0,0,0,0,0,...,0,0,0,0,0,1,1
3,1050613,2018-01-28,1,0,0,0,0,...,0,0,0,0,0,0,1
4,1050614,2018-01-28,0,0,0,0,0,...,0,0,0,0,0,1,1


In [74]:
P1.to_csv('P1.csv')

##3 DATOS COMERCIALES

Tales como:

- Fecha de alta del usuario
- Canal de entrada
- Si es o no activo
- Segmento (que lo hemos añadido tambien a SD1) 

Partimos del dfc y añadiremos variables para la antigüedad, en días y en meses para usar una u otra según nos interese

*(C1)*

In [75]:
dfc.head()

,Unnamed: 0,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
3,3,1050613,2018-01-28,2015-08-10,KHD,0.0,03 - UNIVERSITARIO
4,4,1050614,2018-01-28,2015-08-10,KHE,1.0,03 - UNIVERSITARIO


In [76]:
C1=dfc.copy()

In [77]:
C1.drop('Unnamed: 0', axis=1, inplace = True)

In [78]:
C1.isnull().sum()

pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [79]:
C1['segment'].fillna(value='04 - OTROS', inplace=True)
C1['entry_channel'].fillna(value='DES', inplace=True)

In [80]:
C1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pk_cid           int64  
 1   pk_partition     object 
 2   entry_date       object 
 3   entry_channel    object 
 4   active_customer  float64
 5   segment          object 
dtypes: float64(1), int64(1), object(4)
memory usage: 273.0+ MB


In [81]:
C1['entry_date']= np.where(C1['entry_date'].str.contains('2015-02-29'),'2015-02-28',C1['entry_date'])
C1['entry_date']= np.where(C1['entry_date'].str.contains('2016-02-29'),'2016-02-28',C1['entry_date'])
C1['entry_date']= np.where(C1['entry_date'].str.contains('2017-02-29'),'2017-02-28',C1['entry_date'])
C1['entry_date']= np.where(C1['entry_date'].str.contains('2018-02-29'),'2018-02-28',C1['entry_date'])
C1['entry_date']= np.where(C1['entry_date'].str.contains('2019-02-29'),'2019-02-28',C1['entry_date'])

In [82]:
C1['active_customer'] = C1['active_customer'].astype(bool)
C1['pk_partition']=pd.to_datetime(C1['pk_partition'])
C1['entry_date']=pd.to_datetime(C1['entry_date'])

In [83]:
C1.columns

Index(['pk_cid', 'pk_partition', 'entry_date', 'entry_channel',
       'active_customer', 'segment'],
      dtype='object')

In [84]:
columns_friendly=['ID_USR','FECHA_PARTICION','FECHA_ENTRADA','CANAL_ENTRADA','ACTIVO/A','SEGMENTO']

In [85]:
C1.columns=columns_friendly

In [86]:
C1['ANTIGUEDAD_DIAS'] = (C1['FECHA_PARTICION']-C1['FECHA_ENTRADA']).dt.days
C1['ANTIGUEDAD_MESES'] = C1['FECHA_PARTICION'].dt.to_period('M').view(dtype='int') -\
    C1['FECHA_ENTRADA'].dt.to_period('M').view(dtype='int')

In [87]:
C1.head()

,ID_USR,FECHA_PARTICION,FECHA_ENTRADA,CANAL_ENTRADA,ACTIVO/A,SEGMENTO,ANTIGUEDAD_DIAS,ANTIGUEDAD_MESES
0,1375586,2018-01-28,2018-01-12,KHL,True,02 - PARTICULARES,16,0
1,1050611,2018-01-28,2015-08-10,KHE,False,03 - UNIVERSITARIO,902,29
2,1050612,2018-01-28,2015-08-10,KHE,False,03 - UNIVERSITARIO,902,29
3,1050613,2018-01-28,2015-08-10,KHD,False,03 - UNIVERSITARIO,902,29
4,1050614,2018-01-28,2015-08-10,KHE,True,03 - UNIVERSITARIO,902,29


In [88]:
C1.to_csv('C1.csv')

## CARTERA

Nos interesa analizar conjuntamente algunos datos sociodemográficos y comerciales, cómo es nuestra cartera. 

Crearemos un nuevo dataset agrupando esos datos por la última partición para analizar la situación actual y generaremos otro para observar la evolución


*(C_UR Y C_EVO)*

In [89]:
CARTERA=SD1.copy()


In [90]:
CARTERA['ANTIGUEDAD_DIAS']=C1['ANTIGUEDAD_DIAS']
CARTERA['CANAL_ENTRADA']=C1['CANAL_ENTRADA']
CARTERA['ACTIVO/A']=C1['ACTIVO/A']
CARTERA['TOTAL_PROD']=P1['TOTAL_PROD']
CARTERA['ANTIGUEDAD_MESES']=C1['ANTIGUEDAD_MESES']

In [91]:
#Eliminaremos fallecimientos, ya que no interesan en cartera o acciones comerciales, únciamente como estadística demográfica
CARTERA.drop(CARTERA[CARTERA['FALLECIDO/A']=='S'].index, axis=0 , inplace=True)

In [92]:
columnas_usadas=['PAIS','REGION(ESP)','FALLECIDO/A']
CARTERA.drop(columnas_usadas, axis=1, inplace = True)

In [93]:
CARTERA.head()

,ID_USR,FECHA_PARTICION,SEXO,EDAD,SALARIO,SEGMENTO,RANGO_SALARIAL,DE_ESPAÑA,COM.AUTONOMA_ESP,RANGO_EDAD,ANTIGUEDAD_DIAS,CANAL_ENTRADA,ACTIVO/A,TOTAL_PROD,ANTIGUEDAD_MESES
0,1375586,2018-01-28,M,35,87218.10,02 - PARTICULARES,50.000 a 100.000€,SI,ANDALUCIA,19 a 35,16,KHL,True,1,0
1,1050611,2018-01-28,H,23,35548.74,03 - UNIVERSITARIO,0 a 50.000€,SI,CASTILLA LA MANCHA,19 a 35,902,KHE,False,1,29
2,1050612,2018-01-28,H,23,122179.11,03 - UNIVERSITARIO,100.000 a 150.000€,SI,CASTILLA LA MANCHA,19 a 35,902,KHE,False,1,29
3,1050613,2018-01-28,M,22,119775.54,03 - UNIVERSITARIO,100.000 a 150.000€,SI,ARAGON,19 a 35,902,KHD,False,1,29
4,1050614,2018-01-28,H,23,0.00,03 - UNIVERSITARIO,0 a 50.000€,SI,ARAGON,19 a 35,902,KHE,True,1,29


In [94]:
CARTERA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5961849 entries, 0 to 5962923
Data columns (total 15 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ID_USR            int64         
 1   FECHA_PARTICION   datetime64[ns]
 2   SEXO              object        
 3   EDAD              int64         
 4   SALARIO           float64       
 5   SEGMENTO          object        
 6   RANGO_SALARIAL    category      
 7   DE_ESPAÑA         object        
 8   COM.AUTONOMA_ESP  object        
 9   RANGO_EDAD        category      
 10  ANTIGUEDAD_DIAS   int64         
 11  CANAL_ENTRADA     object        
 12  ACTIVO/A          bool          
 13  TOTAL_PROD        int64         
 14  ANTIGUEDAD_MESES  int64         
dtypes: bool(1), category(2), datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 608.4+ MB


In [95]:
CGB=CARTERA.groupby("FECHA_PARTICION")
#foto fija de la cartera ahora
C_UR=CGB.get_group((CARTERA['FECHA_PARTICION'].max()))

In [96]:
C_UR

,ID_USR,FECHA_PARTICION,SEXO,EDAD,SALARIO,SEGMENTO,RANGO_SALARIAL,DE_ESPAÑA,COM.AUTONOMA_ESP,RANGO_EDAD,ANTIGUEDAD_DIAS,CANAL_ENTRADA,ACTIVO/A,TOTAL_PROD,ANTIGUEDAD_MESES
5519929,657826,2019-05-28,M,44,54493.38,02 - PARTICULARES,50.000 a 100.000€,SI,CATALUÑA,36 a 50,1465,013,True,1,48
5519930,657817,2019-05-28,H,32,0.00,04 - OTROS,0 a 50.000€,SI,CATALUÑA,19 a 35,16,DES,False,0,0
5519931,657986,2019-05-28,M,39,100993.17,02 - PARTICULARES,100.000 a 150.000€,SI,ANDALUCIA,36 a 50,1195,KAF,True,6,39
5519932,657905,2019-05-28,M,85,154059.09,01 - TOP,150.000 a 200.000€,SI,MADRID,> 80,840,KAT,True,2,27
5519933,657336,2019-05-28,H,38,0.00,02 - PARTICULARES,0 a 50.000€,SI,MADRID,36 a 50,61,KAT,True,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,H,22,43912.17,03 - UNIVERSITARIO,0 a 50.000€,SI,ARAGON,19 a 35,1017,KHE,False,1,33
5962920,1166764,2019-05-28,H,23,23334.99,03 - UNIVERSITARIO,0 a 50.000€,SI,LA RIOJA,19 a 35,1017,KHE,False,1,33
5962921,1166763,2019-05-28,M,47,0.00,02 - PARTICULARES,0 a 50.000€,SI,ARAGON,36 a 50,1017,KHE,True,1,33
5962922,1166789,2019-05-28,M,22,199592.82,03 - UNIVERSITARIO,150.000 a 200.000€,SI,ARAGON,19 a 35,1017,KHE,False,1,33


In [97]:
C_EVO=CARTERA.groupby("FECHA_PARTICION").agg(
    TOTAL_USR=("ID_USR",lambda series: len(series[series >= 0])),
    TOTAL_ACTIVOS=("ACTIVO/A",lambda series: len(series[series == True])),
    TOTAL_CLIENTES=("TOTAL_PROD",lambda series: len(series[series == 1])),
    TOTAL_CLIENTES_2mas=("TOTAL_PROD",lambda series: len(series[series > 1]))
).round(0)

In [98]:
C_EVO

,TOTAL_USR,TOTAL_ACTIVOS,TOTAL_CLIENTES,TOTAL_CLIENTES_2mas
FECHA_PARTICION,,,,
2018-01-28,239446,108174,197571,37454
2018-02-28,242474,111048,198975,39002
2018-03-28,245211,113769,199714,40979
2018-04-28,247417,116281,200525,42349
2018-05-28,249879,119029,202360,42990
2018-06-28,252059,121524,202826,44529
2018-07-28,339284,129250,212708,46079
2018-08-28,352865,136314,225787,45955
2018-09-28,375259,144744,239758,48298


In [99]:
C_UR.to_csv('C_UR.csv')
C_EVO.to_csv('C_EVO.csv')

## VENTAS

Datasets puramente comerciales:

- Evolución del total de contratos por producto y global

- Variación de productos contratados, qué clientes contratan y qué clientes anulan

- En base a lo anterior, categorías según status del cliente

*(V1 y P_EVO)*

In [100]:
P_EVO=P1.groupby("FECHA_PARTICION").agg(
    PROD_DEPOSITO_CP = ("PROD_DEPOSITO_CP", np.sum),
    PROD_PRESTAMOS = ("PROD_PRESTAMOS", np.sum),
    PROD_HIPOTECA = ("PROD_HIPOTECA", np.sum),
    PROD_FONDOS = ("PROD_FONDOS", np.sum),
    PROD_VALORES = ("PROD_VALORES", np.sum),
    PROD_DEPOSITOS_LP = ("PROD_DEPOSITOS_LP", np.sum),
    PROD_CUENTA_EMplusplus = ("PROD_CUENTA_EM++", np.sum),
    PROD_TARJETA_CREDITO = ("PROD_TARJETA_CREDITO", np.sum),
    PROD_DOMICILIACIONES = ("PROD_DOMICILIACIONES", np.sum),
    PROD_PLAN_PENSIONES = ("PROD_PLAN_PENSIONES", np.sum),
    PROD_CUENTA_BON_DOMICILIA = ("PROD_CUENTA_BON_DOMICILIA", np.sum),
    PROD_CUENTA_EM_CRYPTO = ("PROD_CUENTA_EM_CRYPTO", np.sum),
    PROD_TARJETA_DEBITO = ("PROD_TARJETA_DEBITO", np.sum),
    PROD_CUENTA_EMplus = ("PROD_CUENTA_EM+", np.sum),
    PROD_CUENTA_EM = ("PROD_CUENTA_EM", np.sum),
    TOTAL_PROD = ("TOTAL_PROD", np.sum)
).round(0)

In [101]:
P_EVO

,PROD_DEPOSITO_CP,PROD_PRESTAMOS,PROD_HIPOTECA,PROD_FONDOS,PROD_VALORES,PROD_DEPOSITOS_LP,PROD_CUENTA_EMplusplus,...,PROD_PLAN_PENSIONES,PROD_CUENTA_BON_DOMICILIA,PROD_CUENTA_EM_CRYPTO,PROD_TARJETA_DEBITO,PROD_CUENTA_EMplus,PROD_CUENTA_EM,TOTAL_PROD
FECHA_PARTICION,,,,,,,,,,,,,,,
2018-01-28,883,19,15,786,932,4884,0,...,8835,13478,15320,24696,2,215293,296613
2018-02-28,1343,19,15,872,965,4956,0,...,9496,14246,15827,25448,2,217098,302458
2018-03-28,1664,23,17,969,965,4965,0,...,9882,14989,16428,27046,2,218683,308879
2018-04-28,1740,24,17,1084,983,5107,0,...,10560,15825,16947,27764,2,219816,313499
2018-05-28,1455,27,18,1145,1012,5356,0,...,10148,16697,17569,27911,2,221291,316422
2018-06-28,1244,27,19,1159,1019,5588,0,...,11635,15540,17950,28828,2,224328,322287
2018-07-28,1282,28,20,1172,1053,5796,0,...,12779,16916,18185,29578,2,234324,337160
2018-08-28,1335,29,20,1219,1133,5923,0,...,12126,18118,18333,29205,2,245980,349012
2018-09-28,1386,31,20,1237,1201,6158,0,...,12624,18862,18618,31684,2,261210,369172


In [102]:
V1=P1.copy()

In [103]:
V1['ID_USR']=V1['ID_USR'].astype(object)

In [104]:
V1['VAR_PROD']=V1.groupby("ID_USR")['TOTAL_PROD'].diff()

In [105]:
V1['VAR_PROD'].fillna(value=(V1['TOTAL_PROD']), inplace=True)

In [106]:
V1['VAR_PROD']=V1['VAR_PROD'].astype(int)

In [107]:
V1['STATUS_CLIENTE']='pdte'
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']>0)&(V1['VAR_PROD']==V1['TOTAL_PROD']),'CLIENTE NUEVO',V1['STATUS_CLIENTE'])
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']>0)&(V1['VAR_PROD']>0)&(V1['VAR_PROD']!=V1['TOTAL_PROD']),'CLIENTE CRECIENTE',V1['STATUS_CLIENTE'])
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']>0)&(V1['VAR_PROD']==0),'CLIENTE FIDELIZADO',V1['STATUS_CLIENTE'])
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']==0)&(V1['VAR_PROD']==0),'CLIENTE POTENCIAL',V1['STATUS_CLIENTE'])
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']>0)&(V1['VAR_PROD']<0),'CLIENTE DESERTOR',V1['STATUS_CLIENTE'])
V1['STATUS_CLIENTE']=np.where((V1['TOTAL_PROD']==0)&(V1['VAR_PROD']<0),'CLIENTE PERDIDO',V1['STATUS_CLIENTE'])

In [108]:
V1

,ID_USR,FECHA_PARTICION,PROD_DEPOSITO_CP,PROD_PRESTAMOS,PROD_HIPOTECA,PROD_FONDOS,PROD_VALORES,...,PROD_CUENTA_EM_CRYPTO,PROD_TARJETA_DEBITO,PROD_CUENTA_EM+,PROD_CUENTA_EM,TOTAL_PROD,VAR_PROD,STATUS_CLIENTE
0,1375586,2018-01-28,0,0,0,0,0,...,0,0,0,1,1,1,CLIENTE NUEVO
1,1050611,2018-01-28,0,0,0,0,0,...,0,0,0,1,1,1,CLIENTE NUEVO
2,1050612,2018-01-28,0,0,0,0,0,...,0,0,0,1,1,1,CLIENTE NUEVO
3,1050613,2018-01-28,1,0,0,0,0,...,0,0,0,0,1,1,CLIENTE NUEVO
4,1050614,2018-01-28,0,0,0,0,0,...,0,0,0,1,1,1,CLIENTE NUEVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,0,0,0,0,0,...,0,0,0,1,1,0,CLIENTE FIDELIZADO
5962920,1166764,2019-05-28,0,0,0,0,0,...,0,0,0,1,1,0,CLIENTE FIDELIZADO
5962921,1166763,2019-05-28,0,0,0,0,0,...,0,0,0,1,1,0,CLIENTE FIDELIZADO
5962922,1166789,2019-05-28,0,0,0,0,0,...,0,0,0,1,1,0,CLIENTE FIDELIZADO


In [109]:
V1[V1['ID_USR']==1050613]

,ID_USR,FECHA_PARTICION,PROD_DEPOSITO_CP,PROD_PRESTAMOS,PROD_HIPOTECA,PROD_FONDOS,PROD_VALORES,...,PROD_CUENTA_EM_CRYPTO,PROD_TARJETA_DEBITO,PROD_CUENTA_EM+,PROD_CUENTA_EM,TOTAL_PROD,VAR_PROD,STATUS_CLIENTE
3,1050613,2018-01-28,1,0,0,0,0,...,0,0,0,0,1,1,CLIENTE NUEVO
472596,1050613,2018-02-28,1,0,0,0,0,...,0,0,0,0,1,0,CLIENTE FIDELIZADO
693180,1050613,2018-03-28,0,0,0,0,0,...,0,0,0,0,0,-1,CLIENTE PERDIDO
1581538,1050613,2018-07-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
2050117,1050613,2018-08-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
2331062,1050613,2018-09-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
2673594,1050613,2018-10-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
3335098,1050613,2018-11-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
3416910,1050613,2018-12-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL
4162354,1050613,2019-01-28,0,0,0,0,0,...,0,0,0,0,0,0,CLIENTE POTENCIAL


In [110]:
V1['STATUS_CLIENTE'].value_counts()

CLIENTE FIDELIZADO    4247679
CLIENTE POTENCIAL     1093959
CLIENTE NUEVO          359574
CLIENTE CRECIENTE      131942
CLIENTE DESERTOR       102222
CLIENTE PERDIDO         27548
Name: STATUS_CLIENTE, dtype: int64

Definiciones status cliente:


CLIENTE POTENCIAL: En el periodo anterior no tenía productos y ahora tampoco

CLIENTE NUEVO:     En el periodo anterior no tenía productos y ahora tiene

CLIENTE FIDELIZADO:En el periodo anterior tenía productos y ahora los mantiene

CLIENTE CRECIENTE: En el periodo anterior tenía productos y ahora tiene más

CLIENTE DESERTOR:  En el periodo anterior tenía productos y ahora tiene menos

CLIENTE PERDIDO:   En el periodo anterior tenía productos y ahora no tiene  

In [111]:
P_EVO.to_csv('P_EVO.csv')
V1.to_csv('V1.csv')

## INFORMES

Presentaremos informes resumen de la información comercial y las ventas más relevantes a nuestro juicio:

- Informe 1: Ventas, Anulaciones, Resultado y Acumulado, por mes
- Informe 2: Lo mismo pero agrupado por status de cliente para saber qué tipo de clientes compran o anulan

*(IFORME1 e INFORME2)*

In [112]:
INFORME1=V1.groupby("FECHA_PARTICION").agg(
    Ventas = ("VAR_PROD", lambda series: np.sum(series[series >= 0])),
    Anulaciones = ("VAR_PROD", lambda series: np.sum(series[series < 0])),
    Resultado = ("VAR_PROD", np.sum),
    Acumulado = ("TOTAL_PROD", np.sum),
).astype(int)

In [113]:
INFORME1

,Ventas,Anulaciones,Resultado,Acumulado
FECHA_PARTICION,,,,
2018-01-28,296613,0,296613,296613
2018-02-28,13638,-7740,5898,302458
2018-03-28,13621,-7150,6471,308879
2018-04-28,12201,-7468,4733,313499
2018-05-28,11868,-8935,2933,316422
2018-06-28,13809,-7800,6009,322287
2018-07-28,23683,-9180,14503,337160
2018-08-28,23231,-11337,11894,349012
2018-09-28,29714,-9520,20194,369172


In [114]:
INFORME2=V1.groupby(["FECHA_PARTICION","STATUS_CLIENTE"]).agg(
    Ventas = ("VAR_PROD", lambda series: np.sum(series[series >= 0])),
    Anulaciones = ("VAR_PROD", lambda series: np.sum(series[series < 0])),
    Resultado = ("VAR_PROD", np.sum),
    Acumulado = ("TOTAL_PROD", np.sum),
).astype(int)

In [115]:
INFORME2


Ventas  Anulaciones  Resultado  Acumulado
FECHA_PARTICION STATUS_CLIENTE                                               
2018-01-28      CLIENTE NUEVO       296613            0     296613     296613
                CLIENTE POTENCIAL        0            0          0          0
2018-02-28      CLIENTE CRECIENTE     8955            0       8955      20033
                CLIENTE DESERTOR         0        -6441      -6441       8714
                CLIENTE FIDELIZADO       0            0          0     269028
...                                    ...          ...        ...        ...
2019-05-28      CLIENTE DESERTOR         0        -8735      -8735      11646
                CLIENTE FIDELIZADO       0            0          0     394541
                CLIENTE NUEVO         4351            0       4351       4351
                CLIENTE PERDIDO          0        -2008      -2008          0
                CLIENTE POTENCIAL        0            0          0          0

[98 rows x 4 columns]

In [116]:
INFORME1.to_csv('INFORME1.csv')
INFORME2.to_csv('INFORME2.csv')

#SEGMENTACIÓN

Se trata de un *clustering* 

Generaremos un dataset específico para esta tarea a partir de los datasets anteriores que se agruparon en la última partición

*CLUST*

In [117]:
C2=C1.copy()

In [118]:
C2.head()

,ID_USR,FECHA_PARTICION,FECHA_ENTRADA,CANAL_ENTRADA,ACTIVO/A,SEGMENTO,ANTIGUEDAD_DIAS,ANTIGUEDAD_MESES
0,1375586,2018-01-28,2018-01-12,KHL,True,02 - PARTICULARES,16,0
1,1050611,2018-01-28,2015-08-10,KHE,False,03 - UNIVERSITARIO,902,29
2,1050612,2018-01-28,2015-08-10,KHE,False,03 - UNIVERSITARIO,902,29
3,1050613,2018-01-28,2015-08-10,KHD,False,03 - UNIVERSITARIO,902,29
4,1050614,2018-01-28,2015-08-10,KHE,True,03 - UNIVERSITARIO,902,29


In [119]:
C2.drop('SEGMENTO', axis=1, inplace = True)
C2.drop('FECHA_ENTRADA', axis=1, inplace = True)

In [120]:
C2GB=C2.groupby("FECHA_PARTICION")
C2_UR=C2GB.get_group((C2['FECHA_PARTICION'].max()))

In [121]:
VGB=V1.groupby("FECHA_PARTICION")
V1_UR=VGB.get_group((V1['FECHA_PARTICION'].max()))

In [122]:
S2GB=SD1.groupby("FECHA_PARTICION")
S2_UR=S2GB.get_group((SD1['FECHA_PARTICION'].max()))

In [123]:
CLUST=S2_UR.merge(C2_UR, on='ID_USR', how='left').merge(V1_UR, on='ID_USR', how='left')

In [124]:
CLUST.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442995 entries, 0 to 442994
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   ID_USR                     442995 non-null  object        
 1   FECHA_PARTICION_x          442995 non-null  datetime64[ns]
 2   PAIS                       442995 non-null  object        
 3   REGION(ESP)                442995 non-null  object        
 4   SEXO                       442995 non-null  object        
 5   EDAD                       442995 non-null  int64         
 6   FALLECIDO/A                442995 non-null  object        
 7   SALARIO                    442995 non-null  float64       
 8   SEGMENTO                   442995 non-null  object        
 9   RANGO_SALARIAL             442995 non-null  category      
 10  DE_ESPAÑA                  442995 non-null  object        
 11  COM.AUTONOMA_ESP           442995 non-null  object  

In [125]:
CLUST.drop(CLUST[CLUST['FALLECIDO/A']=='S'].index, axis=0 , inplace=True)

In [126]:
CLUST['DE_ESPAÑA']=np.where(CLUST['DE_ESPAÑA']=='SI',1,0)

In [127]:
CLUST['ACTIVO/A']=CLUST['ACTIVO/A'].astype(int)

In [128]:
CLUST['SEGMENTO_ID']=OrdinalEncoder().fit_transform(CLUST[['SEGMENTO']]).astype(int)
CLUST['STATUS_ID']=OrdinalEncoder().fit_transform(CLUST[['STATUS_CLIENTE']]).astype(int)
CLUST['CANAL_ENT_ID']=OrdinalEncoder().fit_transform(CLUST[['CANAL_ENTRADA']]).astype(int)
CLUST['CCAA_ID']=OrdinalEncoder().fit_transform(CLUST[['COM.AUTONOMA_ESP']]).astype(int)
CLUST['RANGO_SALARIAL_ID']=OrdinalEncoder().fit_transform(CLUST[['RANGO_SALARIAL']]).astype(int)


In [ ]:
#min_perc=0
#min_inc=(CLUST['CANAL_ENTRADA'].value_counts(normalize=True) > min_perc ).sum()

In [129]:
#setGrouped(CLUST,'CANAL_ENTRADA', min_inc,'Otros')

In [130]:
CLUST['SEXO']=np.where(CLUST['SEXO']=='H',1,CLUST['SEXO'])
CLUST['SEXO']=np.where(CLUST['SEXO']=='M',2,CLUST['SEXO'])
CLUST['SEXO']=np.where(CLUST['SEXO']=='ND',3,CLUST['SEXO'])

In [131]:
CLUST['SEXO'].value_counts()

2    227653
1    215254
3         2
Name: SEXO, dtype: int64

In [132]:
CLUST[CLUST['SEXO']==3]

,ID_USR,FECHA_PARTICION_x,PAIS,REGION(ESP),SEXO,EDAD,FALLECIDO/A,...,VAR_PROD,STATUS_CLIENTE,SEGMENTO_ID,STATUS_ID,CANAL_ENT_ID,CCAA_ID,RANGO_SALARIAL_ID
3262,216507,2019-05-28,ES,28.0,3,73,N,...,0,CLIENTE FIDELIZADO,1,2,61,14,1
4174,476023,2019-05-28,ES,28.0,3,70,N,...,0,CLIENTE FIDELIZADO,1,2,58,14,3


In [133]:
CLUST['SEXO']=np.where(CLUST['ID_USR']==216507,1,CLUST['SEXO'])
CLUST['SEXO']=np.where(CLUST['ID_USR']==476023,2,CLUST['SEXO'])

In [134]:
CLUST['SEXO']=CLUST['SEXO'].astype(int)

In [135]:
eliminar=['FECHA_PARTICION_x', 'PAIS', 'REGION(ESP)', 'RANGO_SALARIAL',
       'RANGO_EDAD', 'FECHA_PARTICION_y','FALLECIDO/A',
       'FECHA_PARTICION',]

In [136]:
CLUST.drop(eliminar, axis=1, inplace = True)

In [137]:
CLUST.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442909 entries, 0 to 442994
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID_USR                     442909 non-null  object 
 1   SEXO                       442909 non-null  int64  
 2   EDAD                       442909 non-null  int64  
 3   SALARIO                    442909 non-null  float64
 4   SEGMENTO                   442909 non-null  object 
 5   DE_ESPAÑA                  442909 non-null  int64  
 6   COM.AUTONOMA_ESP           442909 non-null  object 
 7   CANAL_ENTRADA              442909 non-null  object 
 8   ACTIVO/A                   442909 non-null  int64  
 9   ANTIGUEDAD_DIAS            442909 non-null  int64  
 10  ANTIGUEDAD_MESES           442909 non-null  int64  
 11  PROD_DEPOSITO_CP           442909 non-null  int64  
 12  PROD_PRESTAMOS             442909 non-null  int64  
 13  PROD_HIPOTECA              44

In [138]:
CLUST.columns

Index(['ID_USR', 'SEXO', 'EDAD', 'SALARIO', 'SEGMENTO', 'DE_ESPAÑA',
       'COM.AUTONOMA_ESP', 'CANAL_ENTRADA', 'ACTIVO/A', 'ANTIGUEDAD_DIAS',
       'ANTIGUEDAD_MESES', 'PROD_DEPOSITO_CP', 'PROD_PRESTAMOS',
       'PROD_HIPOTECA', 'PROD_FONDOS', 'PROD_VALORES', 'PROD_DEPOSITOS_LP',
       'PROD_CUENTA_EM++', 'PROD_TARJETA_CREDITO', 'PROD_DOMICILIACIONES',
       'PROD_PLAN_PENSIONES', 'PROD_CUENTA_BON_DOMICILIA',
       'PROD_CUENTA_EM_CRYPTO', 'PROD_TARJETA_DEBITO', 'PROD_CUENTA_EM+',
       'PROD_CUENTA_EM', 'TOTAL_PROD', 'VAR_PROD', 'STATUS_CLIENTE',
       'SEGMENTO_ID', 'STATUS_ID', 'CANAL_ENT_ID', 'CCAA_ID',
       'RANGO_SALARIAL_ID'],
      dtype='object')

In [144]:
CLUST.to_csv('CLUST.csv')

# RECOMENDACIÓN

Necesitamos predecir qué tipo de producto tiene más probabilidades de venderse para el próximo mes, a fin de lanzar una campaña de marketing dirigida a aquella familia de productos que teniendo en cuenta esta probabilidad y la comisión por producto, pueda obtener un mayor retorno.

Generaremos un dataset específico que agrupará buena parte de las variables para trabajar en un modelo que permita predecir la probabilidad.

In [139]:
REC=SD1.merge(V1, how='left')

In [140]:
REC['CANAL_ENTRADA']=C1['CANAL_ENTRADA']
REC['ACTIVO/A']=C1['ACTIVO/A']
REC['ANTIGUEDAD_MESES']=C1['ANTIGUEDAD_MESES']

In [141]:
REC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 34 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   ID_USR                     object        
 1   FECHA_PARTICION            datetime64[ns]
 2   PAIS                       object        
 3   REGION(ESP)                object        
 4   SEXO                       object        
 5   EDAD                       int64         
 6   FALLECIDO/A                object        
 7   SALARIO                    float64       
 8   SEGMENTO                   object        
 9   RANGO_SALARIAL             category      
 10  DE_ESPAÑA                  object        
 11  COM.AUTONOMA_ESP           object        
 12  RANGO_EDAD                 category      
 13  PROD_DEPOSITO_CP           int64         
 14  PROD_PRESTAMOS             int64         
 15  PROD_HIPOTECA              int64         
 16  PROD_FONDOS                int64    

In [142]:
REC.to_csv('REC.csv')

In [143]:
pd.to_pickle(REC,"/content/drive/MyDrive/TFM/data/REC_pck")